# Lab : Batch Updates to a S3 Datalake using Apache Hudi

# Copy on Write 

## Table of Contents:

1. [Overview](#Overview)
2. [Copy On Write](#Copy-On-Write)
   2.1 [Bulk Insert the Initial Dataset](#Bulk-Insert-the-Initial-Dataset)
   2.2 [Batch Upsert some records](#Batch-Upsert-some-records)
   2.3 [Deleting Records](#Deleting-Records.)
3. [Rollback](#Rollback)


## Overview

This notebook demonstrates using PySpark on [Apache Hudi](https://aws.amazon.com/emr/features/hudi/) on Amazon EMR to insert/upsert/delete records to an S3 data lake.

Here is a good reference link to read later:

* [How Hudi Works](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-hudi-how-it-works.html)

This notebook covers the following concepts when writing Copy On Write tables to an S3 Datalake:

- Write Hudi Spark jobs in PySpark.
- Bulk Insert the Initial Dataset.
- Write a Non-Partitioned table.
- Sync the Hudi tables to the Hive/Glue Catalog.
- Upsert some records to a Hudi table.
- Delete records from a Hudi table.
- Rollback.





#### Pre-requisites

### This demo is based on Hudi version 0.5.0-incubating and runs fine on Jupyter Notebooks connected to a 1 node (r5.4xlarge) EMR cluster with configuration listed below 

 - EMR versions 5.29.0 or 6.0.0 
 
 - Software configuration

       - Hadoop 2.8.5
       - Hive 2.3.6
       - Livy 0.6.0
       - JupyterHub 1.0.0
       - Spark 2.4.4
       
       
 - AWS Glue Data Catalog settings - Select the below listed check boxes
       - Use for Hive table metadata  
       - Use for Spark table metadata



### Connect to the Master Node of EMR cluster Using SSH :
    - ssh -i ~/xxxx.pem hadoop@<ec2-xx-xxx-xx-xx.us-west-2.compute.amazonaws.com>

    - Ensure  the below listed files are copied into HDFS.

    - hadoop fs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar hdfs:///user/hadoop/

    - hadoop fs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar hdfs:///user/hadoop/

    - hadoop fs -copyFromLocal /usr/lib/spark/jars/httpclient-4.5.9.jar hdfs:///user/hadoop/

Let's start by initializing the Spark Session to connect this notebook to our Spark EMR cluster:

In [ ]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"hdfs:///user/hadoop/httpclient-4.5.9.jar, hdfs:///user/hadoop/hudi-spark-bundle.jar,hdfs:///user/hadoop/spark-avro.jar",
             "spark.sql.hive.convertMetastoreParquet":"false",     
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             "spark.executor.memory": "7G",
             "spark.executor.cores": 1,
             "spark.dynamicAllocation.initialExecutors":16
           } 
}

In [ ]:
## CHANGE ME ##
config = {
    "table_name": "hudi_trips_table",
    "target": "s3://<Your S3 Bucket Here>/tmp/hudi/hudi_trips_table",
    "primary_key": "trip_id",
    "sort_key": "tstamp",
    "commits_to_retain": "10"
}

The constants for Python to use:

In [ ]:
# General Constants
HUDI_FORMAT = "org.apache.hudi"
TABLE_NAME = "hoodie.table.name"
RECORDKEY_FIELD_OPT_KEY = "hoodie.datasource.write.recordkey.field"
PRECOMBINE_FIELD_OPT_KEY = "hoodie.datasource.write.precombine.field"
OPERATION_OPT_KEY = "hoodie.datasource.write.operation"
BULK_INSERT_OPERATION_OPT_VAL = "bulk_insert"
UPSERT_OPERATION_OPT_VAL = "upsert"
BULK_INSERT_PARALLELISM = "hoodie.bulkinsert.shuffle.parallelism"
UPSERT_PARALLELISM = "hoodie.upsert.shuffle.parallelism"
S3_CONSISTENCY_CHECK = "hoodie.consistency.check.enabled"
HUDI_CLEANER_POLICY = "hoodie.cleaner.policy"
KEEP_LATEST_COMMITS = "KEEP_LATEST_COMMITS"
HUDI_COMMITS_RETAINED = "hoodie.cleaner.commits.retained"
PAYLOAD_CLASS_OPT_KEY = "hoodie.datasource.write.payload.class"
EMPTY_PAYLOAD_CLASS_OPT_VAL = "org.apache.hudi.EmptyHoodieRecordPayload"

# Hive Constants
HIVE_SYNC_ENABLED_OPT_KEY="hoodie.datasource.hive_sync.enable"
HIVE_PARTITION_FIELDS_OPT_KEY="hoodie.datasource.hive_sync.partition_fields"
HIVE_ASSUME_DATE_PARTITION_OPT_KEY="hoodie.datasource.hive_sync.assume_date_partitioning"
HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY="hoodie.datasource.hive_sync.partition_extractor_class"
HIVE_TABLE_OPT_KEY="hoodie.datasource.hive_sync.table"

# Partition Constants
NONPARTITION_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.NonPartitionedExtractor"
MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.MultiPartKeysValueExtractor"
KEYGENERATOR_CLASS_OPT_KEY="hoodie.datasource.write.keygenerator.class"
NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.NonpartitionedKeyGenerator"
COMPLEX_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.ComplexKeyGenerator"
PARTITIONPATH_FIELD_OPT_KEY="hoodie.datasource.write.partitionpath.field"

#Incremental Constants
VIEW_TYPE_OPT_KEY="hoodie.datasource.view.type"
BEGIN_INSTANTTIME_OPT_KEY="hoodie.datasource.read.begin.instanttime"
VIEW_TYPE_INCREMENTAL_OPT_VAL="incremental"
END_INSTANTTIME_OPT_KEY="hoodie.datasource.read.end.instanttime"

Functions to create JSON data and Spark dataframe from this data

In [ ]:
## Generates Data

from datetime import datetime

def get_json_data(start, count, dest):
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data = [{"trip_id": i, "tstamp": time_stamp, "route_id": chr(65 + (i % 10)), "destination": dest[i%10]} for i in range(start, start + count)]
    return data

# Creates the Dataframe
def create_json_df(spark, data):
    sc = spark.sparkContext
    return spark.read.json(sc.parallelize(data))


## Bulk Insert the Initial Dataset

Let's generate 2M records to load into our Data Lake:

In [ ]:
dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
df1 = create_json_df(spark, get_json_data(0, 2000000, dest))
print(df1.count())
df1.show()

## Copy On Write

The default table type is Copy-On-Write which is best suited for read-heavy workloads with modest writes. Copy-On-Write creates commit files with original data + the new changes during writing itself. While this increases latency on writes, this set up makes it more manageable for faster read.


We will be using the Copy on write storage option(STORAGE_TYPE_OPT_KEY, "COPY_ON_WRITE") which is the default option
and need not be explicitly set.

Write the data to S3: 

In [ ]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .mode("Overwrite")
      .save(config['target']))

Let's observe the number of files in S3. Expected number of files is 3 files as BULK_INSERT_PARALLELISM is set to 3. 
```

$ aws s3 ls s3://<Your S3 Bucket Here>/tmp/hudi/hudi_trips_table/ --summarize --human-readable
                           PRE .hoodie/
2020-04-28 23:11:39    0 Bytes .hoodie_$folder$
2020-04-28 23:11:52   93 Bytes .hoodie_partition_metadata
2020-04-28 23:11:59    4.8 MiB 0eeafaf2-0110-4056-b509-724b11a4c0b6-0_0-7-67_20200428231141.parquet
2020-04-28 23:11:59    4.4 MiB 1ce9cbac-56e4-477e-a2bc-33fe62b3550f-0_1-7-68_20200428231141.parquet
2020-04-28 23:11:59    4.6 MiB 579443dc-8f56-4990-bed6-a527f21e9682-0_2-7-69_20200428231141.parquet

Total Objects: 5
   Total Size: 13.8 MiB

```

```

Let's inspect the table created and query the data:

In [ ]:
spark.sql("show tables").show(20,False)

In [ ]:
spark.sql("show create table "+config['table_name']).show(20,False)

In [ ]:
df2=spark.read.format(HUDI_FORMAT).load(config["target"]+"/*")
df2.count()
df2.show(20,False)

We can query the Hive table as well:

In [ ]:
spark.sql("select count(*) from "+config['table_name']).show(20,False)


## Batch Upsert some records

Let's modify a few records:

In [ ]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name'] +" where trip_id between 1000000 and 1000009").show(20,False)

In [ ]:
upsert_dest = ["Boston", "Boston", "Boston", "Boston", "Boston","Boston","Boston","Boston","Boston","Boston"]
df3 = create_json_df(spark, get_json_data(1000000, 10, upsert_dest))
df3.count()

In [ ]:
df3.select("trip_id","route_id","destination","tstamp").show()

We have changed the destination and timestamp for trip IDs 1000000 to 1000010. Now, let's upsert the changes to S3. Note that the operation now is "Upsert" as opposed to BulkInsert for the initial load:

```
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)

```

In [ ]:
(df3.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

In [ ]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id between 999996 and 1000013").show(50,False)



Now lets check the commit

```
$ aws s3 ls s3://<Your S3 Bucket Here>/tmp/hudi/hudi_trips_table/ --summarize --human-readable
                           PRE .hoodie/
2020-04-28 23:11:39    0 Bytes .hoodie_$folder$
2020-04-28 23:11:52   93 Bytes .hoodie_partition_metadata
2020-04-28 23:15:43    4.8 MiB 0eeafaf2-0110-4056-b509-724b11a4c0b6-0_0-51-609_20200428231528.parquet
2020-04-28 23:11:59    4.8 MiB 0eeafaf2-0110-4056-b509-724b11a4c0b6-0_0-7-67_20200428231141.parquet
2020-04-28 23:11:59    4.4 MiB 1ce9cbac-56e4-477e-a2bc-33fe62b3550f-0_1-7-68_20200428231141.parquet
2020-04-28 23:11:59    4.6 MiB 579443dc-8f56-4990-bed6-a527f21e9682-0_2-7-69_20200428231141.parquet

Total Objects: 6
   Total Size: 18.6 MiB

```

Now that we upserted some records, let us try to insert 10 new records into the table. We will use same upsert option. As primary keys 2000000 to 2000009 do not exist in the table, the records will be inserted. 

In [ ]:
insert_dest = ["Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse"]
df5 = create_json_df(spark, get_json_data(2000000, 10, insert_dest))
df5.count()
df5.show()

In [ ]:
(df5.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

In [ ]:
df6=spark.read.format(HUDI_FORMAT).load(config["target"]+"/*")
df6.count()

In [ ]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id > 1999996").show(50,False)

We can observe that the records are updated.

## Deleting Records.

Apache Hudi supports implementing two types of deletes on data stored in Hudi datasets, by enabling the user to specify a different record payload implementation.

* **Soft Deletes** : With soft deletes, user wants to retain the key but just null out the values for all other fields. This can be simply achieved by ensuring the appropriate fields are nullable in the dataset schema and simply upserting the dataset after setting these fields to null.
    
* **Hard Deletes** : A stronger form of delete is to physically remove any trace of the record from the dataset. 

Let's now execute some hard delete operations on our dataset which will remove the records from our dataset.

Let's delete the 10 records with the "Syracuse" destination we added to the table. Note that the only change is the single line that set the hoodie.datasource.write.payload.class to org.apache.hudi.EmptyHoodieRecordPayload to delete the records.

```
.option(PAYLOAD_CLASS_OPT_KEY, EMPTY_PAYLOAD_CLASS_OPT_VAL)
```

In [ ]:
df5.show()

In [ ]:
(df5.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .option(PAYLOAD_CLASS_OPT_KEY, EMPTY_PAYLOAD_CLASS_OPT_VAL)
      .mode("Append")
      .save(config['target']))

In [ ]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id > 1999996").show(20,False)

We can observe that the records > 2000000 no longer exist in our table.

Let's observe the number of files in S3. Expected : 6 files (initial files (3) + one upsert + one insert + one delete = 6)

```

$ aws s3 ls s3://<Your S3 Bucket Here>/tmp/hudi/hudi_trips_table/ --summarize --human-readable
                           PRE .hoodie/
2020-04-28 23:11:39    0 Bytes .hoodie_$folder$
2020-04-28 23:11:52   93 Bytes .hoodie_partition_metadata
2020-04-28 23:15:43    4.8 MiB 0eeafaf2-0110-4056-b509-724b11a4c0b6-0_0-51-609_20200428231528.parquet
2020-04-28 23:11:59    4.8 MiB 0eeafaf2-0110-4056-b509-724b11a4c0b6-0_0-7-67_20200428231141.parquet
2020-04-28 23:18:24    4.4 MiB 1ce9cbac-56e4-477e-a2bc-33fe62b3550f-0_0-135-1376_20200428231814.parquet
2020-04-28 23:17:17    4.4 MiB 1ce9cbac-56e4-477e-a2bc-33fe62b3550f-0_0-93-1007_20200428231705.parquet
2020-04-28 23:11:59    4.4 MiB 1ce9cbac-56e4-477e-a2bc-33fe62b3550f-0_1-7-68_20200428231141.parquet
2020-04-28 23:11:59    4.6 MiB 579443dc-8f56-4990-bed6-a527f21e9682-0_2-7-69_20200428231141.parquet

Total Objects: 8
   Total Size: 27.5 MiB

```

In our example, we set number of commits to retain as 10. So, maximum only 10 new files can be created on top of our bulk insert files. i.e., 13 files in total. If we had set the commits_to_retain as 2, the number of files created will not increase beyond initial files(3) + commits_to_retain(2) = 5 files. This is because Hudi Cleaning Policy will delete older files when writing based on the commit retain policy.

## Rollback

Let's say we want to roll back the last delete we made. 

Start the hudi CLI using the command-> 
    /usr/lib/hudi/cli/bin/hudi-cli.sh

connect to the hudi/hudi_trips_table using the command ->  
    connect --path s3://<Your S3 Bucket Here>/tmp/hudi/hudi_trips_table/

List the commits using command ->> 
    commits show

hudi:hudi_trips_table->commits show
20/04/28 23:46:09 INFO s3n.S3NativeFileSystem: Opening 's3://<Your S3 Bucket Here>/tmp/hudi/hudi_trips_table/.hoodie/20200428231705.commit' for reading
20/04/28 23:46:09 INFO s3n.S3NativeFileSystem: Opening 's3://<Your S3 Bucket Here>/tmp/hudi/hudi_trips_table/.hoodie/20200428231528.commit' for reading
20/04/28 23:46:09 INFO s3n.S3NativeFileSystem: Opening 's3://<Your S3 Bucket Here>/tmp/hudi/hudi_trips_table/.hoodie/20200428231141.commit' for reading
╔════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime     │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠════════════════╪═════════════════════╪═══════════════════╪═════════════════════╪══════════════════════════╪═══════════════════════╪══════════════════════════════╪══════════════╣
║ 20200428231814 │ 4.4 MB              │ 0                 │ 1                   │ 1                        │ 642132                │ 0                            │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20200428231528 │ 4.8 MB              │ 0                 │ 1                   │ 1                        │ 697329                │ 10                           │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20200428231528 │ 4.8 MB              │ 0                 │ 1                   │ 1                        │ 697329                │ 10                           │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢

║ 20200428231141 │ 13.8 MB             │ 3                 │ 0                   │ 1                        │ 2000000               │ 0                            │ 0            ║
╚════════════════╧═════════════════════╧═══════════════════╧═════════════════════╧══════════════════════════╧═══════════════════════╧══════════════════════════════╧══════════════╝

commit rollback --commit 20200428231814
    
Now let us check what happened to the records we deleted earlier.

In [ ]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id > 1999996").show(20,False)